<a href="https://colab.research.google.com/github/skywalker0803r/mxnet_course/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install mxnet-cu100 d2lzh
import d2lzh as d2l
import mxnet as mx
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata,nn
import os,sys

# GPU 狀態

In [2]:
!nvidia-smi

Wed Dec  4 07:13:48 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 建造Alexnet

In [0]:
net = nn.Sequential()
# Conv => relu => pool
net.add(nn.Conv2D(96,kernel_size=11,strides=4,activation='relu'))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
# Conv => relu => pool
net.add(nn.Conv2D(256,kernel_size=5,padding=2,activation='relu'))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
# (Conv => relu) *3 => pool
net.add(nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'))
net.add(nn.Conv2D(384,kernel_size=3,padding=1,activation='relu'))
net.add(nn.Conv2D(256,kernel_size=3,padding=1,activation='relu'))
net.add(nn.MaxPool2D(pool_size=3,strides=2))
# (Dense 4096 => relu => Dropout 0.5) * 2
net.add(nn.Dense(4096,activation="relu"))
net.add(nn.Dropout(0.5))
net.add(nn.Dense(4096,activation="relu"))
net.add(nn.Dropout(0.5))
# output 10
net.add(nn.Dense(10))

In [4]:
#觀察每一層輸出形狀
X = nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name,'output shape{}'.format(X.shape))

conv0 output shape(1, 96, 54, 54)
pool0 output shape(1, 96, 26, 26)
conv1 output shape(1, 256, 26, 26)
pool1 output shape(1, 256, 12, 12)
conv2 output shape(1, 384, 12, 12)
conv3 output shape(1, 384, 12, 12)
conv4 output shape(1, 256, 12, 12)
pool2 output shape(1, 256, 5, 5)
dense0 output shape(1, 4096)
dropout0 output shape(1, 4096)
dense1 output shape(1, 4096)
dropout1 output shape(1, 4096)
dense2 output shape(1, 10)


# 讀取數據集


In [5]:
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size=128,resize=224)

# 訓練模型

In [0]:
#初始化權重
net.initialize(init=init.Xavier(),ctx=mx.gpu(),force_reinit=True)
#梯度下降方法
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.1})
#定義loss
loss = gluon.loss.SoftmaxCrossEntropyLoss()
#開始訓練
d2l.train(train_iter,test_iter,net,loss,trainer,ctx=mx.gpu(),num_epochs=10)